<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/270_MissionOrchestratorAgent_CompletionCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Completion Check Node

In [ ]:
def completion_check_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Completion Check Node: Check if mission is complete.

    This node determines if the mission is complete by checking:
    1. All tasks are completed
    2. All required approvals are granted
    3. No tasks remaining in queue

    This is a conditional node - routes based on completion status.

    Input:
        - tasks_completed (int): Number of completed tasks
        - tasks_total (int): Total number of tasks
        - task_queue (List[Dict]): Remaining tasks
        - pending_approvals (List[Dict]): Tasks awaiting approval
        - mission_status (str): Current mission status

    Output:
        - mission_status (str): Updated status ("completed" or "in_progress" or "awaiting_approval")
        - completion_reason (Optional[str]): Why mission completed/failed
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    tasks_completed = state.get("tasks_completed", 0)
    tasks_total = state.get("tasks_total", 0)
    task_queue = state.get("task_queue", [])
    pending_approvals = state.get("pending_approvals", [])
    mission_status = state.get("mission_status", "in_progress")

    try:
        # Check if all tasks are completed
        all_tasks_completed = tasks_completed >= tasks_total and len(task_queue) == 0

        # Check if all approvals are granted
        all_approvals_granted = len(pending_approvals) == 0

        # Determine completion status
        if all_tasks_completed and all_approvals_granted:
            mission_status = "completed"
            completion_reason = "All tasks completed and all approvals granted"
        elif all_tasks_completed and not all_approvals_granted:
            mission_status = "awaiting_approval"
            completion_reason = f"All tasks completed but {len(pending_approvals)} approval(s) pending"
        elif not all_tasks_completed:
            # Still in progress
            completion_reason = None
        else:
            completion_reason = None

        return {
            "mission_status": mission_status,
            "completion_reason": completion_reason,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"completion_check_node: Unexpected error: {str(e)}"]
        }


def report_generation_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Report Generation Node: Generate and save final mission report.

    This node:
    1. Generates comprehensive markdown report
    2. Saves report to file
    3. Returns report content and file path

    Input:
        - All state fields (for comprehensive report)
        - mission_id (str): Mission identifier

    Output:
        - mission_report (str): Generated markdown report
        - report_file_path (Optional[str]): Path to saved report file
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    mission_id = state.get("mission_id")
    config = MissionOrchestratorConfig()

    if not mission_id:
        return {
            "errors": errors + ["report_generation_node: mission_id is required"]
        }

    try:
        # Generate report
        report_content = generate_mission_report(state)

        # Save report
        report_file_path = save_report(
            report_content,
            mission_id,
            reports_dir=config.reports_dir
        )

        return {
            "mission_report": report_content,
            "report_file_path": report_file_path,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"report_generation_node: Unexpected error: {str(e)}"]
        }



# 🎯 NODE 1: `completion_check_node`

This is the “Are we done yet?” node.

Imagine the agent as a kid cleaning their room.
This node is the parent asking:

> “Okay, are you ACTUALLY finished?”

This node looks at **three things**:

1️⃣ Did you finish all your tasks?
2️⃣ Are there any tasks left in the queue?
3️⃣ Do you still need approval for anything?

If all the answers are:

* yes, tasks done
* yes, queue empty
* yes, approvals done

👉 Mission is **complete**.

Let’s break it down.

---

# ⭐ Step-by-Step (Kid Friendly)

### 1. Get important stuff from the backpack (`state`):

* how many tasks were completed
* how many tasks total
* what’s left in the task queue
* any pending approvals
* current mission status (in_progress / awaiting_approval / completed)

---

### 2. Check if all tasks are completed

```python
all_tasks_completed = tasks_completed >= tasks_total and len(task_queue) == 0
```

Kid version:

> “Did I finish ALL my chores? Is my to-do list empty?”

---

### 3. Check if all approvals are granted

```python
all_approvals_granted = len(pending_approvals) == 0
```

Kid version:

> “Did the teacher sign off on EVERYTHING?”

---

### 4. Decide the mission status

This is the logic:

### ⭐ Case A — All tasks done + all approvals done

Mission = **completed**

```python
mission_status = "completed"
completion_reason = "All tasks completed and all approvals granted"
```

---

### ⭐ Case B — All tasks done but approvals pending

Mission = **awaiting_approval**

```python
mission_status = "awaiting_approval"
completion_reason = "All tasks completed but X approval(s) pending"
```

Kid version:

> “I cleaned my room, but my parent still needs to check it.”

---

### ⭐ Case C — Not all tasks done

Mission = **in_progress**

Kid version:

> “Still cleaning!”

---

### 5. Return updated status

The node returns:

* mission_status
* completion_reason
* errors

This is a **routing node** —
it decides where the workflow goes next.

---

# 🌟 Why This Node Matters

Without a completion check:

* the agent wouldn’t know when to stop
* it wouldn’t know when to wait
* it wouldn’t know when to resume
* it wouldn’t know when to generate the final report
* it wouldn’t know when it’s truly done

It’s the **traffic cop** at the end of the workflow.

And it is one of the simplest but most essential nodes.

---

# 🎯 NODE 2: `report_generation_node`

This is the “Write your final book report” node.
The agent has finished its adventure and now writes everything it did.

This node:

1️⃣ Generates a markdown report
2️⃣ Saves it to a file
3️⃣ Returns the content + file path

Super clean. Super simple. Super powerful.

---

# 🔥 Step-by-Step (Kid Version)

### 1. Get mission_id from backpack

```python
mission_id = state.get("mission_id")
```

Kid version:

> “I need your name to write on the report.”

If mission_id is missing → return an error.

---

### 2. Generate the report

```python
report_content = generate_mission_report(state)
```

This is where the agent:

* shares its progress
* lists all tasks
* shows KPI improvements
* lists approvals
* lists errors
* creates a polished Markdown summary

This is like writing the final essay.

---

### 3. Save the report

```python
report_file_path = save_report(...)
```

This creates a file with a timestamp, like:

```
output/mission_reports/mission_report_M001_20240212_153000.md
```

Kid version:

> “Turn in your finished homework and save a copy in your binder.”

---

### 4. Return results

The node returns:

* the markdown text (mission_report)
* where the file was saved (report_file_path)

This is the FINAL node in your workflow.

---

# 🌟 Why This Node Is AMAZING

It provides:

### ✔ Full transparency

Exactly what the agent did, step by step.

### ✔ Trust

Humans can read everything the agent completed.

### ✔ Documentation

Perfect for audits, debugging, and historical logs.

### ✔ Business value

Shows improvement %, KPI gains, time saved.

### ✔ Proof of performance

Companies LOVE visible results.

### ✔ Closure

The mission ends cleanly and properly.

---

# 🎉 PUTTING IT ALL TOGETHER

Your workflow looks like this:

1. **Planning Nodes**
2. **Data Loading Nodes**
3. **Task Ordering Node**
4. **Task Execution Node(s)**
5. **Progress Tracking Node**
6. **Approval Check Node**
7. **Completion Check Node**
8. **Report Generation Node**

This is **real orchestration**.
This is **enterprise architecture**.
This is **a complete agent life cycle**.



# Standalone test script for reporting and completion checking

In [ ]:
"""Standalone test script for reporting and completion checking"""

from agents.mission_orchestrator.utilities.report_generation import (
    generate_mission_report,
    save_report
)
from agents.mission_orchestrator.nodes import completion_check_node, report_generation_node
from agents.mission_orchestrator.utilities.data_loading import (
    load_mission_tasks,
    load_mission_kpis
)
from config import MissionOrchestratorState
from datetime import datetime, timedelta


def test_report_generation_utilities():
    """Test report generation utilities"""
    print("=" * 60)
    print("Testing Report Generation Utilities")
    print("=" * 60)

    # Create sample state
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "mission": {
            "mission_id": "M001",
            "mission_name": "Reduce Customer Onboarding Time",
            "description": "Optimize steps required to onboard new customers to shorten time-to-value."
        },
        "goal": {
            "objective": "Execute mission M001",
            "mission_id": "M001"
        },
        "executed_tasks": [
            {
                "task_id": "T1",
                "task": "Collect customer information",
                "status": "completed",
                "agent_name": "Data Collection Agent",
                "duration_minutes": 5.0,
                "requires_approval": False
            },
            {
                "task_id": "T2",
                "task": "Verify documents",
                "status": "completed",
                "agent_name": "Document Verification Agent",
                "duration_minutes": 10.0,
                "requires_approval": True
            },
            {
                "task_id": "T3",
                "task": "Schedule onboarding call",
                "status": "completed",
                "agent_name": "Scheduling Agent",
                "duration_minutes": 3.0,
                "requires_approval": False
            }
        ],
        "mission_kpis": load_mission_kpis("M001"),
        "kpi_metrics": {
            "actual_onboarding_time_days": 0.01,
            "actual_steps": 3,
            "improvement_percentage": 99.8
        },
        "kpi_status": {
            "onboarding_time": "exceeded",
            "steps": "on_track"
        },
        "approval_history": [
            {
                "task_id": "T2",
                "decision": "approved",
                "decided_by": "auto_approval",
                "decided_at": datetime.now().isoformat()
            }
        ],
        "pending_approvals": [],
        "tasks_completed": 3,
        "tasks_total": 3,
        "progress_percentage": 100.0,
        "elapsed_time_minutes": 18.0,
        "mission_status": "completed",
        "errors": []
    }

    # Test 1: Generate report
    print("\n1. Generating mission report...")
    report = generate_mission_report(state)
    print(f"   ✓ Report generated ({len(report)} characters)")
    print(f"\n   Preview (first 500 characters):")
    print("   " + "-" * 56)
    print("   " + "\n   ".join(report[:500].split("\n")[:10]))
    print("   " + "...")

    # Test 2: Save report
    print("\n2. Saving report to file...")
    filepath = save_report(report, "M001")
    print(f"   ✓ Report saved to: {filepath}")

    # Verify file exists
    from pathlib import Path
    if Path(filepath).exists():
        file_size = Path(filepath).stat().st_size
        print(f"   ✓ File verified ({file_size} bytes)")

    print("\n" + "=" * 60)
    print("Report Generation Utilities Tests Complete!")
    print("=" * 60)


def test_completion_check_node():
    """Test completion check node"""
    print("\n" + "=" * 60)
    print("Testing Completion Check Node")
    print("=" * 60)

    # Test 1: Mission complete (all tasks done, all approvals granted)
    print("\n1. Testing completed mission...")
    state: MissionOrchestratorState = {
        "tasks_completed": 3,
        "tasks_total": 3,
        "task_queue": [],
        "pending_approvals": [],
        "mission_status": "in_progress",
        "errors": []
    }

    result = completion_check_node(state)
    print(f"   ✓ Mission status: {result['mission_status']}")
    print(f"   Completion reason: {result.get('completion_reason', 'N/A')}")

    # Test 2: Mission awaiting approval
    print("\n2. Testing mission awaiting approval...")
    state = {
        "tasks_completed": 3,
        "tasks_total": 3,
        "task_queue": [],
        "pending_approvals": [
            {"task_id": "T2", "task": "Verify documents", "status": "pending"}
        ],
        "mission_status": "in_progress",
        "errors": []
    }

    result = completion_check_node(state)
    print(f"   ✓ Mission status: {result['mission_status']}")
    print(f"   Completion reason: {result.get('completion_reason', 'N/A')}")

    # Test 3: Mission in progress
    print("\n3. Testing mission in progress...")
    state = {
        "tasks_completed": 2,
        "tasks_total": 3,
        "task_queue": [{"task_id": "T3"}],
        "pending_approvals": [],
        "mission_status": "in_progress",
        "errors": []
    }

    result = completion_check_node(state)
    print(f"   ✓ Mission status: {result['mission_status']}")
    print(f"   Completion reason: {result.get('completion_reason', 'N/A')}")

    print("\n" + "=" * 60)
    print("Completion Check Node Test Complete!")
    print("=" * 60)


def test_report_generation_node():
    """Test report generation node"""
    print("\n" + "=" * 60)
    print("Testing Report Generation Node")
    print("=" * 60)

    # Setup complete state
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "mission": {
            "mission_id": "M001",
            "mission_name": "Reduce Customer Onboarding Time",
            "description": "Optimize steps required to onboard new customers."
        },
        "goal": {
            "objective": "Execute mission M001",
            "mission_id": "M001"
        },
        "executed_tasks": [
            {
                "task_id": "T1",
                "task": "Collect customer information",
                "status": "completed",
                "agent_name": "Data Collection Agent",
                "duration_minutes": 5.0,
                "requires_approval": False
            },
            {
                "task_id": "T2",
                "task": "Verify documents",
                "status": "completed",
                "agent_name": "Document Verification Agent",
                "duration_minutes": 10.0,
                "requires_approval": True
            },
            {
                "task_id": "T3",
                "task": "Schedule onboarding call",
                "status": "completed",
                "agent_name": "Scheduling Agent",
                "duration_minutes": 3.0,
                "requires_approval": False
            }
        ],
        "mission_kpis": load_mission_kpis("M001"),
        "kpi_metrics": {
            "actual_onboarding_time_days": 0.01,
            "actual_steps": 3,
            "improvement_percentage": 99.8
        },
        "kpi_status": {
            "onboarding_time": "exceeded",
            "steps": "on_track"
        },
        "approval_history": [
            {
                "task_id": "T2",
                "decision": "approved",
                "decided_by": "auto_approval",
                "decided_at": datetime.now().isoformat()
            }
        ],
        "tasks_completed": 3,
        "tasks_total": 3,
        "progress_percentage": 100.0,
        "elapsed_time_minutes": 18.0,
        "mission_status": "completed",
        "errors": []
    }

    result = report_generation_node(state)

    if "mission_report" in result:
        print("   ✓ Report generated successfully")
        print(f"   - Report length: {len(result['mission_report'])} characters")
        print(f"   - File path: {result.get('report_file_path', 'N/A')}")

        # Show preview
        report_lines = result['mission_report'].split('\n')[:15]
        print(f"\n   Report preview:")
        for line in report_lines:
            print(f"     {line}")
        print("     ...")
    else:
        print("   ✗ Report generation failed")
        if result.get("errors"):
            for error in result["errors"]:
                print(f"     - {error}")

    print("\n" + "=" * 60)
    print("Report Generation Node Test Complete!")
    print("=" * 60)


def test_full_flow_with_reporting():
    """Test full flow including completion check and reporting"""
    print("\n" + "=" * 60)
    print("Testing Full Flow with Completion Check and Reporting")
    print("=" * 60)

    from agents.mission_orchestrator.nodes import (
        goal_node, planning_node, data_loading_node,
        task_ordering_node, task_execution_node,
        approval_check_node, progress_tracking_node,
        completion_check_node, report_generation_node
    )

    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Step 1-4: Setup
    print("\n1-4. Setting up mission...")
    goal_result = goal_node(state)
    state = {**state, **goal_result}

    planning_result = planning_node(state)
    state = {**state, **planning_result}

    data_result = data_loading_node(state)
    state = {**state, **data_result}

    ordering_result = task_ordering_node(state)
    state = {**state, **ordering_result}

    # Step 5: Execute all tasks
    print("\n5. Executing tasks...")
    max_iterations = 10
    iteration = 0

    while state.get("task_queue") and iteration < max_iterations:
        iteration += 1

        # Execute task
        execution_result = task_execution_node(state)
        state = {**state, **execution_result}

        # Check approvals
        approval_result = approval_check_node(state)
        state = {**state, **approval_result}

        # Track progress
        progress_result = progress_tracking_node(state)
        state = {**state, **progress_result}

        if not state.get("task_queue"):
            break

    # Step 6: Check completion
    print("\n6. Checking mission completion...")
    completion_result = completion_check_node(state)
    state = {**state, **completion_result}
    print(f"   Mission status: {state['mission_status']}")
    if state.get('completion_reason'):
        print(f"   Reason: {state['completion_reason']}")

    # Step 7: Generate report
    if state['mission_status'] == "completed":
        print("\n7. Generating final report...")
        report_result = report_generation_node(state)
        state = {**state, **report_result}

        if "mission_report" in state:
            print(f"   ✓ Report generated and saved")
            print(f"   File: {state.get('report_file_path', 'N/A')}")

            # Show summary from report
            report_lines = state['mission_report'].split('\n')
            print(f"\n   Report Summary:")
            for line in report_lines[:8]:
                if line.strip():
                    print(f"     {line}")
        else:
            print("   ✗ Report generation failed")

    print(f"\n--- Final Mission Status ---")
    print(f"  Status: {state['mission_status']}")
    print(f"  Tasks: {state['tasks_completed']}/{state['tasks_total']}")
    print(f"  Progress: {state.get('progress_percentage', 0):.1f}%")
    print(f"  Report generated: {'Yes' if 'mission_report' in state else 'No'}")

    print("\n" + "=" * 60)
    print("Full Flow with Reporting Test Complete!")
    print("=" * 60)


if __name__ == "__main__":
    try:
        test_report_generation_utilities()
        test_completion_check_node()
        test_report_generation_node()
        test_full_flow_with_reporting()
        print("\n✅ All tests completed successfully!")
    except Exception as e:
        print(f"\n❌ Error during testing: {e}")
        import traceback
        traceback.print_exc()



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent % python test_reporting_standalone.py
============================================================
Testing Report Generation Utilities
============================================================

1. Generating mission report...
   ✓ Report generated (1291 characters)

   Preview (first 500 characters):
   --------------------------------------------------------
   # Mission Execution Report

   **Mission:** Reduce Customer Onboarding Time
   **Mission ID:** M001
   **Generated:** 2025-12-11 17:11:53

   ---

   ## Executive Summary

   ...

2. Saving report to file...
   ✓ Report saved to: output/mission_reports/mission_report_M001_20251211_171153.md
   ✓ File verified (1299 bytes)

============================================================
Report Generation Utilities Tests Complete!
============================================================

============================================================
Testing Completion Check Node
============================================================

1. Testing completed mission...
   ✓ Mission status: completed
   Completion reason: All tasks completed and all approvals granted

2. Testing mission awaiting approval...
   ✓ Mission status: awaiting_approval
   Completion reason: All tasks completed but 1 approval(s) pending

3. Testing mission in progress...
   ✓ Mission status: in_progress
   Completion reason: None

============================================================
Completion Check Node Test Complete!
============================================================

============================================================
Testing Report Generation Node
============================================================
   ✓ Report generated successfully
   - Report length: 1266 characters
   - File path: output/mission_reports/mission_report_M001_20251211_171153.md

   Report preview:
     # Mission Execution Report

     **Mission:** Reduce Customer Onboarding Time
     **Mission ID:** M001
     **Generated:** 2025-12-11 17:11:53

     ---

     ## Executive Summary

     **Status:** completed
     **Progress:** 100.0%
     **Tasks Completed:** 3/3
     **Elapsed Time:** 18.00 minutes

     ...

============================================================
Report Generation Node Test Complete!
============================================================

============================================================
Testing Full Flow with Completion Check and Reporting
============================================================

1-4. Setting up mission...

5. Executing tasks...

6. Checking mission completion...
   Mission status: awaiting_approval
   Reason: All tasks completed but 1 approval(s) pending

--- Final Mission Status ---
  Status: awaiting_approval
  Tasks: 3/3
  Progress: 100.0%
  Report generated: No

============================================================
Full Flow with Reporting Test Complete!
============================================================

✅ All tests completed successfully!
